<a href="https://colab.research.google.com/github/subpic/ava-mlsp/blob/master/predict_mlsp_wide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### download libraries and model

In [1]:
%%capture 
!pip install kuti
!git clone https://github.com/subpic/ava-mlsp.git
!wget -O ava-mlsp/models/irnv2_mlsp_wide_orig/model_best_weights.h5 https://www.dropbox.com/s/16k0vh1dn7ls0cd/model_best_weights.h5?dl=1&raw=1
!pip install munch

What follows is a trivially modified version of ava-mlsp/predict_mlsp_wide.ipynb:

In [2]:
import kuti
from kuti import model_helper as mh
from kuti import applications as apps
from kuti import tensor_ops as ops
from kuti import generic as gen
from kuti import image_utils as img
import pandas as pd, numpy as np, os

Loaded Kuti


In [3]:
root_path = '/content/ava-mlsp/' # CHANGED to reflect PATH on colab

dataset = root_path + 'metadata/AVA_data_official_test.csv';
images_path = root_path + 'images/'
ids = pd.read_csv(dataset)

In [4]:
# define the MLSP wide model (updated from kuti.applications)
from kuti.applications import InceptionResNetV2, Lambda, tf, Concatenate, Model,\
Input, GlobalAveragePooling2D

def model_inceptionresnet_pooled(input_shape=(None, None, 3), indexes=list(range(43)),
                                 pool_size=(5, 5), name='', return_sizes=False):
    """
    Returns the wide MLSP features, spatially pooled, from InceptionResNetV2.
    * input_shape: shape of the input images
    * indexes: indices of the modules to use
    * pool_size: spatial extend of the MLSP features
    * name: name of the model
    * return_sizes: return the sizes of each layer: (model, pool_sizes)
    :return: model or (model, pool_sizes)
    """

    print('Loading InceptionResNetV2 multi-pooled with input_shape:', input_shape)
    model_base = InceptionResNetV2(weights     = 'imagenet',
                                   include_top = False,
                                   input_shape = input_shape)
    print('Creating multi-pooled model')

    ImageResizer = Lambda(lambda x: tf.image.resize(x, pool_size),
                          name='feature_resizer')

    feature_layers = [l for l in model_base.layers if 'mixed' in l.name]
    feature_layers = [feature_layers[i] for i in indexes]
    pools = [ImageResizer(l.output) for l in feature_layers]
    conc_pools = Concatenate(name='conc_pools', axis=3)(pools)

    model = Model(inputs  = model_base.input, outputs = conc_pools)
    if name: model.name = name

    if return_sizes:
        pool_sizes = [[np.int32(x) for x in f.get_shape()[1:]] for f in pools]
        return model, pool_sizes
    else:
        return model

### Load combined model

In [5]:
# load base model
model_name = 'mlsp_wide_orig'
input_shape = (None, None, 3)
model_base = model_inceptionresnet_pooled(input_shape)
pre = apps.process_input[apps.InceptionResNetV2]

# MODEL DEF
input_feats = Input(shape=(5,5,16928), dtype='float32')
x = apps.inception_block(input_feats, size=1024)
x = GlobalAveragePooling2D(name='final_GAP')(x)

pred = apps.fc_layers(x, name       = 'head',
                      fc_sizes      = [2048, 1024, 256,  1],
                      dropout_rates = [0.25, 0.25, 0.5, 0],
                      batch_norm    = 2)

model = Model(inputs  = input_feats, 
              outputs = pred)

gen_params = dict(batch_size    = 1,
                  data_path     = images_path,                  
                  process_fn    = pre,
                  input_shape   = input_shape,
                  inputs        = 'image_name',
                  outputs       = 'MOS', 
                  fixed_batches = False)

helper = mh.ModelHelper(model, model_name, ids, 
                        gen_params = gen_params)

# load head model
helper.load_model(model_name = root_path + 'models/irnv2_mlsp_wide_orig/model')

# join base and head models
helper.model = Model(inputs  = model_base.input, 
                     outputs = model(model_base.output))

Loading InceptionResNetV2 multi-pooled with input_shape: (None, None, 3)
219070464/219055592 [==============================] - 1s 0us/step
Creating multi-pooled model
Model weights loaded: /content/ava-mlsp/models/irnv2_mlsp_wide_orig/model_best_weights.h5


### Predict the score of a single image

In [6]:
# choose an image from the test set
image_path = images_path + ids[ids.set == 'test'].iloc[0].image_name

# load, pre-process it, and pass it to the model
I = pre( img.read_image(image_path) )
I = np.expand_dims(I, 0)
I_score = helper.model.predict(I)

print('predicted image score:', I_score[0][0])

predicted image score: 6.543146
